In [22]:
from ultralytics.data.utils import polygon2mask, visualize_image_annotations
from ultralytics.data.annotator import auto_annotate
import cv2
import numpy as np
from pathlib import Path
import random
from PIL import Image
from ultralytics.utils.ops import segments2boxes

In [17]:
n_aug=5
src=Path("../qa/QA_01/lid/")
out=Path("../qa_aug")
pseg=Path("../qa_seg")
bg=Path("../qa_aug_bg")
out.mkdir(parents=True, exist_ok=True)

In [ ]:
auto_annotate(data=src, det_model="../best.pt", sam_model="sam2.1_l.pt", classes=[1], device="cpu", output_dir=pseg)

In [18]:
def read_seg(ipath, imgsz):
    lmask=[]
    with open(pseg/(ipath.with_suffix(".txt").name), "r") as f:
        for line in f:
            cont=line.split()
            if cont:
                ncls, seg = cont[0], cont[1:]
                # if (not ncls=="1"):
                #     continue
                seg = np.array(seg).astype(float).reshape((-1,2))
                lmask.append(
                    polygon2mask(
                        imgsz,
                        [seg*imgsz[::-1]],
                        color=1,
                        downsample_ratio=1,
                    )
                )
    return sorted(lmask, key=lambda m: np.sum(m))[-1]


def blend_img(img, bgimg, mask):
    imgsz=img.shape[:2]
    bgsz=bgimg.shape[:2]
    mask=cv2.GaussianBlur(mask, (0,0), sigmaX=4, sigmaY=4)
    bg_reshape=(cv2.resize(bgimg,(np.array(bgsz)[::-1]*np.max(np.array(imgsz)/np.array(bgsz))).astype(int))[:imgsz[0],:imgsz[1]])
    return (img.T*mask.T+bg_reshape.T*(1-mask.T)).T


def read_label(ipath):
    bbox=(0,None)
    with open(LABEL_PATH/(ipath.with_suffix(".txt").name), "r") as f:
        for line in f:
            cont = line.split()
            if cont:
                ncls, ibbox=cont[0], cont[1:]
                if ncls=="1":
                    assert len(ibbox)==4
                    bbox=max(bbox, (np.prod(np.array(ibbox[-2:]).astype(float)), ibbox))
    return bbox[-1]

In [35]:
imgs=list(src.glob("*.jpg"))
bimgs=list(bg.glob("*.jpg"))

for pbimg in bimgs:
    bimg=cv2.imread(pbimg)
    for iidx, pimg in enumerate(np.random.choice(imgs, size=n_aug)):
        img=cv2.imread(pimg)
        mask=read_seg(pimg, img.shape[:2])
        # crop to bb, resize
        spr=cv2.resize((img.T*mask.T).T, np.array(img.shape[1::-1])//2)
        top, left, bottom, right=np.argmax(spr.sum(axis=-1).sum(axis=-1)!=0), np.argmax(spr.sum(axis=-1).sum(axis=0)!=0), np.argmax(spr.sum(axis=-1).sum(axis=-1)[::-1]!=0), np.argmax(spr.sum(axis=-1).sum(axis=0)[::-1]!=0)
        spr=spr[top:-bottom,left:-right,::]
        # plot on random spot
        rimg=bimg.copy()
        offset_x=random.randint(0,rimg.shape[0]-spr.shape[0])
        offset_y=random.randint(0,200)
        idx,idy=np.where(spr.sum(axis=-1)!=0)
        rimg[offset_x:offset_x+spr.shape[0],offset_y:offset_y+spr.shape[1],::][(idx,idy)]=spr[(idx,idy)]
        fn=out/f"QA_01_{pbimg.stem}_{iidx}.jpg"
        print(fn)
        cv2.imwrite(fn, rimg)
        

../qa_aug/QA_01_VID_20250210_171732_00023_0.jpg
../qa_aug/QA_01_VID_20250210_171732_00023_1.jpg
../qa_aug/QA_01_VID_20250210_171732_00023_2.jpg
../qa_aug/QA_01_VID_20250210_171732_00023_3.jpg
../qa_aug/QA_01_VID_20250210_171732_00023_4.jpg
../qa_aug/QA_01_VID_20250210_171636_00015_0.jpg
../qa_aug/QA_01_VID_20250210_171636_00015_1.jpg
../qa_aug/QA_01_VID_20250210_171636_00015_2.jpg
../qa_aug/QA_01_VID_20250210_171636_00015_3.jpg
../qa_aug/QA_01_VID_20250210_171636_00015_4.jpg
../qa_aug/QA_01_VID_20250210_171636_00012_0.jpg
../qa_aug/QA_01_VID_20250210_171636_00012_1.jpg
../qa_aug/QA_01_VID_20250210_171636_00012_2.jpg
../qa_aug/QA_01_VID_20250210_171636_00012_3.jpg
../qa_aug/QA_01_VID_20250210_171636_00012_4.jpg
../qa_aug/QA_01_VID_20250210_171732_00058_0.jpg
../qa_aug/QA_01_VID_20250210_171732_00058_1.jpg
../qa_aug/QA_01_VID_20250210_171732_00058_2.jpg
../qa_aug/QA_01_VID_20250210_171732_00058_3.jpg
../qa_aug/QA_01_VID_20250210_171732_00058_4.jpg
../qa_aug/QA_01_VID_20250210_171732_0002

KeyboardInterrupt: 